# Sunbird RC Authentication and Workflow Tutorial

This notebook provides a comprehensive walkthrough of the Sunbird RC system, covering authentication, entity identification (OSID), claims management, and the complete workflow.

## Topics Covered
1. Authentication and Authorization
2. Entity Identification (OSID)
3. Claims Management and Attestation
4. Complete Entity Lifecycle Workflow
5. API Usage Examples
6. Configuration and Deployment

## 1. Authentication and Authorization

### Overview

Sunbird RC uses Keycloak as its Identity and Access Management (IAM) provider. The system supports three authentication flows:

1. Client Credentials Flow - for service accounts and automated operations
2. Resource Owner Password Flow - for end-user authentication
3. Authorization Code Flow - for web applications

### Keycloak Configuration

- Realm Name: sunbird-rc
- Realm URL: http://keycloak:8080/auth/realms/sunbird-rc
- Access Token Lifespan: 300 seconds (5 minutes)
- SSO Session Idle Timeout: 1800 seconds (30 minutes)
- Database: PostgreSQL (shared with registry)

### Keycloak Clients

#### 1. admin-api
- Purpose: Administrative API access and automated operations
- Authentication Type: client-secret
- Grant Types: client_credentials
- Service Account: service-account-admin-api
- Roles: admin

#### 2. registry-frontend
- Purpose: Web UI authentication
- Authentication Type: client-secret
- Grant Types: authorization_code, refresh_token
- Redirect URIs: Web UI application URLs

#### 3. admin-portal
- Purpose: Admin portal authentication
- Authentication Type: public (no secret required)
- Grant Types: authorization_code, implicit
- Redirect URIs: http://localhost:3001/*, http://localhost:3001
- Web Origins: http://localhost:3001

### Getting Authentication Tokens

#### Token Endpoint
POST http://localhost:8080/auth/realms/sunbird-rc/protocol/openid-connect/token

#### Client Credentials Flow
Used for service accounts to obtain tokens for API operations.

Request Parameters:
- grant_type: client_credentials
- client_id: admin-api
- client_secret: <YOUR_CLIENT_SECRET>

Response Includes:
- access_token: JWT token for API requests
- expires_in: Token lifespan in seconds (300)
- token_type: Bearer
- scope: profile email

In [1]:
import requests
import json
from typing import Dict, Optional

class KeycloakClient:
    """Client for interacting with Keycloak authentication."""
    
    def __init__(self, base_url: str = "http://localhost:8080", realm: str = "sunbird-rc"):
        self.base_url = base_url
        self.realm = realm
        self.token_endpoint = f"{base_url}/auth/realms/{realm}/protocol/openid-connect/token"
        self.access_token = None
    
    def get_token(self, client_id: str, client_secret: str) -> str:
        """Get access token using client credentials flow.
        
        Args:
            client_id: The client ID (e.g., 'admin-api')
            client_secret: The client secret
        
        Returns:
            Access token string
        """
        payload = {
            'grant_type': 'client_credentials',
            'client_id': client_id,
            'client_secret': client_secret
        }
        
        response = requests.post(self.token_endpoint, data=payload)
        
        if response.status_code != 200:
            raise Exception(f"Token request failed: {response.text}")
        
        response_data = response.json()
        self.access_token = response_data['access_token']
        return self.access_token
    
    def get_headers(self) -> Dict[str, str]:
        """Get headers with authorization token for API requests."""
        if not self.access_token:
            raise Exception("No access token available. Call get_token() first.")
        
        return {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json'
        }

# Example usage
keycloak_client = KeycloakClient()

# To get a token, use:
# token = keycloak_client.get_token(
#     client_id='admin-api',
#     client_secret='your-secret-here'
# )

print("KeycloakClient class defined successfully")
print(f"Token Endpoint: {keycloak_client.token_endpoint}")

KeycloakClient class defined successfully
Token Endpoint: http://localhost:8080/auth/realms/sunbird-rc/protocol/openid-connect/token


### JWT Token Structure

JWT tokens contain three parts separated by dots: header.payload.signature

#### Token Payload Claims

The payload contains important information about the user and their permissions:

Standard Claims:
- exp: Token expiration time (Unix timestamp)
- iat: Token issued at time (Unix timestamp)
- sub: Subject (user ID)
- aud: Audience (intended client ID)
- iss: Issuer (Keycloak realm URL)

Custom Claims:
- name: User's full name
- email: User's email address
- realm_access: Contains realm-level roles
- resource_access: Contains client-specific roles
- entity: Custom attribute for entity types
- consent: Custom attribute for consent tracking

In [2]:
import base64
import json
from datetime import datetime

def decode_jwt(token: str) -> Dict:
    """Decode JWT token to inspect claims (without verification).
    
    Warning: This only decodes the token, it does not verify the signature.
    Always verify tokens on the server side.
    """
    parts = token.split('.')
    if len(parts) != 3:
        raise ValueError("Invalid JWT format")
    
    # Decode the payload (add padding if necessary)
    payload = parts[1]
    padding = 4 - len(payload) % 4
    if padding != 4:
        payload += '=' * padding
    
    decoded = base64.urlsafe_b64decode(payload)
    return json.loads(decoded)

def print_token_info(claims: Dict) -> None:
    """Print formatted token information."""
    print("Token Claims:")
    print("-" * 50)
    
    # Basic claims
    print(f"Subject (User ID): {claims.get('sub', 'N/A')}")
    print(f"Username: {claims.get('preferred_username', 'N/A')}")
    print(f"Name: {claims.get('name', 'N/A')}")
    print(f"Email: {claims.get('email', 'N/A')}")
    
    # Token timing
    issued_at = datetime.fromtimestamp(claims['iat'])
    expires_at = datetime.fromtimestamp(claims['exp'])
    print(f"Issued at: {issued_at}")
    print(f"Expires at: {expires_at}")
    
    # Authorization
    print(f"Audience (Client): {claims.get('aud', 'N/A')}")
    
    realm_roles = claims.get('realm_access', {}).get('roles', [])
    if realm_roles:
        print(f"Realm Roles: {', '.join(realm_roles)}")
    
    # Custom claims
    if 'entity' in claims:
        print(f"Entities: {claims.get('entity', [])}")

# Example token structure (not a real token)
example_claims = {
    'sub': 'user-123-abc',
    'preferred_username': 'john.doe',
    'name': 'John Doe',
    'email': 'john@example.com',
    'iat': int(datetime.now().timestamp()),
    'exp': int(datetime.now().timestamp()) + 300,
    'aud': 'registry-frontend',
    'realm_access': {'roles': ['default-roles-sunbird-rc', 'admin']},
    'entity': ['Teacher', 'School']
}

print("Example Token Claims:")
print_token_info(example_claims)

Example Token Claims:
Token Claims:
--------------------------------------------------
Subject (User ID): user-123-abc
Username: john.doe
Name: John Doe
Email: john@example.com
Issued at: 2025-12-09 11:30:53
Expires at: 2025-12-09 11:35:53
Audience (Client): registry-frontend
Realm Roles: default-roles-sunbird-rc, admin
Entities: ['Teacher', 'School']


## 2. Entity Identification (OSID)

### What is OSID?

OSID (OpenSearch ID) is the unique identifier for each entity and its nested properties in the Sunbird RC registry. Every entity and every nested object receives an auto-generated OSID.

### OSID Format

OSIDs follow the pattern: {version}-{uuid}

Examples:
- 1-b4907dc2-d3a8-49dc-a933-2b473bdd2ddb (entity root)
- 1-096cd663-6ba9-49f8-af31-1ace9e31bc31 (nested object)
- 1-8d6dfb25-7789-44da-a6d4-eacf93e3a7bb (nested property)

### OSID Purposes

1. Primary Key: Unique identifier in database
2. References: Link between related entities
3. Audit Tracking: Track changes to specific entities
4. Ownership: Identify who owns/created the entity
5. Attestation: Link claims to specific entities

In [3]:
import uuid
from typing import Any, Dict

class OSIDGenerator:
    """Generate and manage OSID (OpenSearch ID) identifiers."""
    
    OSID_VERSION = "1"
    
    @staticmethod
    def generate() -> str:
        """Generate a new OSID."""
        return f"{OSIDGenerator.OSID_VERSION}-{uuid.uuid4()}"
    
    @staticmethod
    def is_valid(osid: str) -> bool:
        """Check if a string is a valid OSID."""
        parts = osid.split('-', 1)
        if len(parts) != 2:
            return False
        
        version = parts[0]
        uuid_part = parts[1]
        
        try:
            uuid.UUID(uuid_part)
            return version in ['1', '2']
        except ValueError:
            return False

class Entity:
    """Represents a Sunbird RC entity with OSID."""
    
    def __init__(self, entity_type: str, data: Dict[str, Any]):
        self.entity_type = entity_type
        self.osid = OSIDGenerator.generate()
        self.data = data
        self.os_owner = None  # Set by system
    
    def to_dict(self) -> Dict:
        """Convert entity to dictionary format for API."""
        entity_data = {
            'osid': self.osid,
            **self.data
        }
        
        if self.os_owner:
            entity_data['osOwner'] = self.os_owner
        
        return {self.entity_type: entity_data}

# Example: Create a Teacher entity
teacher_data = {
    'name': 'John Doe',
    'email': 'john@example.com',
    'phone': '9000090000',
    'qualifications': [
        {
            'degree': 'B.Tech',
            'university': 'IIT Delhi',
            'year': 2015
        }
    ]
}

teacher = Entity('Teacher', teacher_data)
print("Generated Teacher Entity:")
print(json.dumps(teacher.to_dict(), indent=2))
print(f"\nOSID: {teacher.osid}")
print(f"Valid OSID: {OSIDGenerator.is_valid(teacher.osid)}")

Generated Teacher Entity:
{
  "Teacher": {
    "osid": "1-ad3680c4-96c0-41e5-8b44-5097603c4da1",
    "name": "John Doe",
    "email": "john@example.com",
    "phone": "9000090000",
    "qualifications": [
      {
        "degree": "B.Tech",
        "university": "IIT Delhi",
        "year": 2015
      }
    ]
  }
}

OSID: 1-ad3680c4-96c0-41e5-8b44-5097603c4da1
Valid OSID: True


### OSID in API Requests

#### Getting Entity by OSID
GET /api/v1/{EntityType}/{osid}

Example:
GET /api/v1/Teacher/1-b4907dc2-d3a8-49dc-a933-2b473bdd2ddb

#### Updating Entity by OSID
PUT /api/v1/{EntityType}/{osid}

Example:
PUT /api/v1/Teacher/1-b4907dc2-d3a8-49dc-a933-2b473bdd2ddb

#### Deleting Entity by OSID
DELETE /api/v1/{EntityType}/{osid}

Example:
DELETE /api/v1/Teacher/1-b4907dc2-d3a8-49dc-a933-2b473bdd2ddb

## 3. Claims Management and Attestation

### Overview

Claims are requests for verification of entity data by authorized attestors. When an entity property is updated and that property requires attestation (per attestation policy), a claim is automatically created.

### Claim States

Claims progress through two states:

1. OPEN: Claim is created and awaiting attestor review
2. CLOSED: Claim has been processed (approved or rejected)

### Claim Lifecycle

1. Entity Property Updated
   - User updates entity property that requires attestation

2. Attestation Policy Evaluated
   - System checks if property matches attestation policy
   - Verifies property is marked for attestation

3. Claim Created
   - New claim record created with status OPEN
   - Contains reference to entity OSID
   - Contains property path and new value
   - Contains attestor entity type

4. Attestor Notified
   - Attestor entity receives notification
   - Can view pending claims

5. Attestor Reviews and Approves
   - Attestor reviews entity data
   - Makes approval/rejection decision
   - Adds optional notes

6. Claim Closure
   - Claim status changed to CLOSED
   - Attestation recorded with timestamp
   - Attestor info logged for audit

7. Credential Generation
   - If all required attestations complete
   - Verifiable credential (VC) is generated
   - Signed with issuer's private key
   - QR code created for sharing

In [5]:
from enum import Enum
from datetime import datetime
from typing import Optional, List

class ClaimStatus(Enum):
    """Enum representing claim states."""
    OPEN = "OPEN"
    CLOSED = "CLOSED"

class Claim:
    """Represents an attestation claim."""
    
    def __init__(self, entity_id: str, entity_type: str, 
                 property_path: str, attestor_entity: str):
        self.id = str(uuid.uuid4())
        self.entity_id = entity_id
        self.entity_type = entity_type
        self.property_path = property_path
        self.attestor_entity = attestor_entity
        self.status = ClaimStatus.OPEN
        self.created_at = datetime.now()
        self.attested_on = None
        self.attestor_user_id = None
        self.notes = []
    
    def add_note(self, note: str) -> None:
        """Add a note to the claim."""
        self.notes.append({
            'text': note,
            'timestamp': datetime.now()
        })
    
    def attest(self, attestor_user_id: str, approved: bool = True) -> None:
        """Mark claim as attested.
        
        Args:
            attestor_user_id: ID of the attestor
            approved: Whether claim was approved or rejected
        """
        self.status = ClaimStatus.CLOSED
        self.attested_on = datetime.now()
        self.attestor_user_id = attestor_user_id
    
    def to_dict(self) -> Dict:
        """Convert claim to dictionary format."""
        return {
            'id': self.id,
            'entityId': self.entity_id,
            'entityType': self.entity_type,
            'propertyPath': self.property_path,
            'attestorEntity': self.attestor_entity,
            'status': self.status.value,
            'createdAt': self.created_at.isoformat(),
            'attestedOn': self.attested_on.isoformat() if self.attested_on else None,
            'attestorUserId': self.attestor_user_id,
            'notes': self.notes
        }

# Example: Create and process a claim
claim = Claim(
    entity_id='1-b4907dc2-d3a8-49dc-a933-2b473bdd2ddb',
    entity_type='Teacher',
    property_path='$.Teacher.qualifications[0]',
    attestor_entity='EducationBoard'
)

print("New Claim (OPEN):")
print(json.dumps(claim.to_dict(), indent=2))

# Process the claim
claim.add_note('Degree verified from university records')
claim.attest(attestor_user_id='attestor-123')

print("\nProcessed Claim (CLOSED):")
print(json.dumps(claim.to_dict(), indent=2))

New Claim (OPEN):
{
  "id": "a9db0264-3e63-4c4c-8865-dbce9a838d64",
  "entityId": "1-b4907dc2-d3a8-49dc-a933-2b473bdd2ddb",
  "entityType": "Teacher",
  "propertyPath": "$.Teacher.qualifications[0]",
  "attestorEntity": "EducationBoard",
  "status": "OPEN",
  "createdAt": "2025-12-09T11:32:00.957284",
  "attestedOn": null,
  "attestorUserId": null,
  "notes": []
}

Processed Claim (CLOSED):


TypeError: Object of type datetime is not JSON serializable

### Attestation Policy

Attestation policies define which properties require attestation and who can attest them.

Policy Components:
- Name: Human-readable name
- Property: JSONPath to property requiring attestation
- AttestorEntity: Entity type authorized to attest
- Conditions: Optional conditions for policy application

Example Policy:
Property: $.Teacher.qualifications[*]
AttestorEntity: EducationBoard
Meaning: Education Board must attest any qualification added by Teacher

### Claims API Endpoints

The Claims Microservice (port 8082) provides these endpoints:

#### GET /claims
List all claims in the system.
Response: List of claim objects

#### GET /claims/{claimId}
Get details of a specific claim.
Response: Single claim object

#### GET /claims/attestor/{attestorEntity}
Get claims for a specific attestor entity.
Query Parameters: page, size, sort
Response: Paginated list of claims

#### POST /claims/{claimId}/attestation
Process a claim (approve or reject).
Request Body:
- attestorInfo: Information about attestor
- notes: Optional notes
Response: Updated claim object with status CLOSED

## 4. Complete Entity Lifecycle Workflow

This section describes the complete flow from entity creation through credential verification.

### Step 1: Schema Definition
Admin defines entity schemas in the registry. Schemas specify properties, types, and validation rules.

### Step 2: Authentication
User authenticates with Keycloak and receives JWT token.

### Step 3: Entity Creation
User creates entity with initial data. System auto-generates OSID for entity and all nested objects.

### Step 4: Entity Modification
User updates entity properties. System detects changes and evaluates state transitions.

### Step 5: Attestation Policy Evaluation
System checks if modified properties require attestation per defined policies.

### Step 6: Claim Generation
If attestation required, system creates OPEN claim with reference to entity OSID.

### Step 7: Attestor Notification
Attestor entity receives notification of pending claim.

### Step 8: Attestor Review
Attestor reviews entity data and makes approval/rejection decision.

### Step 9: Claim Attestation
Attestor submits attestation, claim status changed to CLOSED.

### Step 10: Credential Generation
System generates W3C-compliant verifiable credential (VC) containing attested claims.

### Step 11: Public Verification
Third parties can scan QR code or enter credential ID to verify authenticity.

In [6]:
class WorkflowState:
    """Represents the state of an entity in the workflow."""
    
    STATES = ['DRAFT', 'SUBMITTED', 'ATTESTED', 'PUBLISHED', 'REVOKED']
    
    def __init__(self):
        self.current_state = 'DRAFT'
        self.history = []
    
    def transition(self, new_state: str, reason: str = '') -> bool:
        """Attempt to transition to a new state."""
        if new_state not in self.STATES:
            return False
        
        self.history.append({
            'from': self.current_state,
            'to': new_state,
            'timestamp': datetime.now(),
            'reason': reason
        })
        
        self.current_state = new_state
        return True
    
    def get_history(self) -> List[Dict]:
        """Get state transition history."""
        return self.history

class EntityWorkflow:
    """Represents complete workflow of an entity."""
    
    def __init__(self, entity_id: str, entity_type: str):
        self.entity_id = entity_id
        self.entity_type = entity_type
        self.state = WorkflowState()
        self.claims = []
        self.credential = None
    
    def add_claim(self, claim: Claim) -> None:
        """Add claim to entity."""
        self.claims.append(claim)
    
    def attest_all_claims(self, attestor_id: str) -> None:
        """Attest all pending claims."""
        for claim in self.claims:
            if claim.status == ClaimStatus.OPEN:
                claim.attest(attestor_id, approved=True)
    
    def generate_credential(self) -> Dict:
        """Generate verifiable credential if all claims attested."""
        if not all(c.status == ClaimStatus.CLOSED for c in self.claims):
            raise Exception("Not all claims are attested")
        
        self.credential = {
            'context': ['https://www.w3.org/2018/credentials/v1'],
            'type': ['VerifiableCredential'],
            'issuer': f'http://registry:8081',
            'issuanceDate': datetime.now().isoformat(),
            'credentialSubject': {
                'id': self.entity_id,
                'type': self.entity_type
            }
        }
        
        self.state.transition('PUBLISHED', 'Credential generated and verified')
        return self.credential

# Example workflow
workflow = EntityWorkflow(
    entity_id='1-b4907dc2-d3a8-49dc-a933-2b473bdd2ddb',
    entity_type='Teacher'
)

print("Initial State:", workflow.state.current_state)

# Simulate workflow
workflow.state.transition('SUBMITTED', 'Entity created')
print("After Creation:", workflow.state.current_state)

# Add a claim
claim = Claim(
    entity_id=workflow.entity_id,
    entity_type=workflow.entity_type,
    property_path='$.Teacher.qualifications[0]',
    attestor_entity='EducationBoard'
)
workflow.add_claim(claim)
print(f"Claim Added: {claim.id}")
print(f"Claim Status: {claim.status.value}")

# Process claim
workflow.attest_all_claims('attestor-123')
print(f"After Attestation: {workflow.claims[0].status.value}")

# Generate credential
credential = workflow.generate_credential()
print(f"\nCredential Generated:")
print(f"State: {workflow.state.current_state}")
print(f"Subject Type: {credential['credentialSubject']['type']}")

Initial State: DRAFT
After Creation: SUBMITTED
Claim Added: 94b04cab-875d-4bd5-8e02-72d82f3a1268
Claim Status: OPEN
After Attestation: CLOSED

Credential Generated:
State: PUBLISHED
Subject Type: Teacher


## 5. API Usage Examples

This section provides practical examples of using the Sunbird RC APIs.

### 5.1 Complete User Journey

This example walks through the complete flow of creating an entity, triggering a claim, and generating a credential.

In [7]:
class SunbirdRCClient:
    """Client for interacting with Sunbird RC APIs."""
    
    def __init__(self, registry_url: str = "http://localhost:8081",
                 keycloak_url: str = "http://localhost:8080",
                 claims_url: str = "http://localhost:8082"):
        self.registry_url = registry_url
        self.keycloak_url = keycloak_url
        self.claims_url = claims_url
        self.session = requests.Session()
        self.access_token = None
    
    def authenticate(self, client_id: str, client_secret: str) -> str:
        """Authenticate and get access token."""
        token_endpoint = f"{self.keycloak_url}/auth/realms/sunbird-rc/protocol/openid-connect/token"
        
        payload = {
            'grant_type': 'client_credentials',
            'client_id': client_id,
            'client_secret': client_secret
        }
        
        response = self.session.post(token_endpoint, data=payload)
        response.raise_for_status()
        
        self.access_token = response.json()['access_token']
        return self.access_token
    
    def _get_headers(self) -> Dict[str, str]:
        """Get headers with authorization."""
        return {
            'Authorization': f'Bearer {self.access_token}',
            'Content-Type': 'application/json'
        }
    
    def create_entity(self, entity_type: str, entity_data: Dict) -> Dict:
        """Create a new entity.
        
        Returns:
            Response containing entity with auto-generated OSID
        """
        url = f"{self.registry_url}/api/v1/{entity_type}"
        payload = {entity_type: entity_data}
        
        response = self.session.post(
            url,
            json=payload,
            headers=self._get_headers()
        )
        response.raise_for_status()
        return response.json()
    
    def update_entity(self, entity_type: str, osid: str, updates: Dict) -> Dict:
        """Update an entity. May trigger claim creation.
        
        Returns:
            Response with updated entity
        """
        url = f"{self.registry_url}/api/v1/{entity_type}/{osid}"
        payload = {entity_type: updates}
        
        response = self.session.put(
            url,
            json=payload,
            headers=self._get_headers()
        )
        response.raise_for_status()
        return response.json()
    
    def get_claims_for_attestor(self, attestor_entity: str, 
                                 page: int = 0, size: int = 10) -> Dict:
        """Get pending claims for an attestor."""
        url = f"{self.claims_url}/claims/attestor/{attestor_entity}"
        params = {'page': page, 'size': size}
        
        response = self.session.get(
            url,
            params=params,
            headers=self._get_headers()
        )
        response.raise_for_status()
        return response.json()
    
    def attest_claim(self, claim_id: str, attestor_info: Dict,
                     notes: str = '') -> Dict:
        """Submit attestation for a claim.
        
        Changes claim status from OPEN to CLOSED
        """
        url = f"{self.claims_url}/claims/{claim_id}/attestation"
        payload = {
            'attestorInfo': attestor_info
        }
        
        if notes:
            payload['notes'] = notes
        
        response = self.session.post(
            url,
            json=payload,
            headers=self._get_headers()
        )
        response.raise_for_status()
        return response.json()
    
    def get_entity(self, entity_type: str, osid: str) -> Dict:
        """Get entity by OSID."""
        url = f"{self.registry_url}/api/v1/{entity_type}/{osid}"
        
        response = self.session.get(
            url,
            headers=self._get_headers()
        )
        response.raise_for_status()
        return response.json()

# Example usage outline (requires running services)
print("Example: Complete User Journey")
print("=" * 50)
print()
print("Step 1: Initialize client")
print("  client = SunbirdRCClient()")
print()
print("Step 2: Authenticate")
print("  token = client.authenticate('admin-api', 'secret')")
print()
print("Step 3: Create Teacher entity")
print("  teacher_data = {")
print("    'name': 'John Doe',")
print("    'email': 'john@example.com',")
print("    'qualifications': [{'degree': 'B.Tech', 'year': 2015}]")
print("  }")
print("  response = client.create_entity('Teacher', teacher_data)")
print("  osid = response['result']['Teacher']['osid']")
print()
print("Step 4: Update qualifications (triggers claim)")
print("  updates = {'qualifications': [{'degree': 'M.Tech', 'year': 2017}]}")
print("  client.update_entity('Teacher', osid, updates)")
print()
print("Step 5: Get pending claims for attestor")
print("  claims = client.get_claims_for_attestor('EducationBoard')")
print()
print("Step 6: Attest the claim")
print("  attestor_info = {")
print("    'osid': 'attestor-osid',")
print("    'name': 'Board Member',")
print("    'entity': 'EducationBoard'")
print("  }")
print("  client.attest_claim(claim_id, attestor_info, notes='Verified')")

Example: Complete User Journey

Step 1: Initialize client
  client = SunbirdRCClient()

Step 2: Authenticate
  token = client.authenticate('admin-api', 'secret')

Step 3: Create Teacher entity
  teacher_data = {
    'name': 'John Doe',
    'email': 'john@example.com',
    'qualifications': [{'degree': 'B.Tech', 'year': 2015}]
  }
  response = client.create_entity('Teacher', teacher_data)
  osid = response['result']['Teacher']['osid']

Step 4: Update qualifications (triggers claim)
  updates = {'qualifications': [{'degree': 'M.Tech', 'year': 2017}]}
  client.update_entity('Teacher', osid, updates)

Step 5: Get pending claims for attestor
  claims = client.get_claims_for_attestor('EducationBoard')

Step 6: Attest the claim
  attestor_info = {
    'osid': 'attestor-osid',
    'name': 'Board Member',
    'entity': 'EducationBoard'
  }
  client.attest_claim(claim_id, attestor_info, notes='Verified')


### 5.2 API Endpoint Reference

#### Registry API (Port 8081)

Entity Operations:
- POST /api/v1/{entity_type} - Create entity
- GET /api/v1/{entity_type}/{osid} - Get entity
- PUT /api/v1/{entity_type}/{osid} - Update entity
- DELETE /api/v1/{entity_type}/{osid} - Delete entity
- GET /api/v1/{entity_type}/search - Search entities

Schema Operations:
- POST /api/v1/Schema - Create schema
- GET /api/v1/Schema/{schema_id} - Get schema
- PUT /api/v1/Schema/{schema_id} - Update schema
- GET /_schemas - List all schemas

Attestation:
- GET /api/v1/attestation-policies - Get policies
- POST /api/v1/{entity_type}/{osid}/attestation - Request attestation

#### Claims API (Port 8082)

- GET /claims - List all claims
- GET /claims/{claim_id} - Get claim details
- GET /claims/attestor/{entity_type} - Get claims for attestor
- POST /claims/{claim_id}/attestation - Attest claim

## 6. Configuration and Deployment

### Environment Variables

#### Keycloak Configuration
- sunbird_sso_url: Keycloak base URL
- sunbird_sso_realm: Realm name
- sunbird_sso_client_id: Client ID for frontend
- sunbird_sso_admin_client_id: Admin client ID
- sunbird_sso_admin_client_secret: Admin client secret

#### Registry Configuration
- authentication_enabled: Enable/disable authentication
- AUTHENTICATION_ENABLED: Alternative name for above
- oauth2_resource_uri: OAuth2 resource endpoint
- oauth2_resource_roles_path: Path to roles in token

#### Claims Configuration
- claims_enabled: Enable/disable claims
- claims_url: Claims service URL

#### Default Passwords
- sunbird_keycloak_user_password: Default password for new users
- KEYCLOAK_ADMIN_PASSWORD: Admin password

### Production Deployment Checklist

Security:
- Change all default passwords
- Enable HTTPS/SSL for all services
- Configure firewall rules
- Enable rate limiting

Operations:
- Set up database backups
- Configure monitoring and alerting
- Enable audit logging
- Set up log aggregation

Data:
- Configure encryption for sensitive data
- Set up data retention policies
- Enable data validation

High Availability:
- Configure multiple Keycloak instances
- Set up load balancing
- Configure database replication
- Use external cache (Redis)

In [8]:
import os
from typing import Dict, Optional

class SunbirdRCConfig:
    """Configuration management for Sunbird RC."""
    
    # Default values
    DEFAULTS = {
        'REGISTRY_URL': 'http://localhost:8081',
        'KEYCLOAK_URL': 'http://localhost:8080',
        'CLAIMS_URL': 'http://localhost:8082',
        'KEYCLOAK_REALM': 'sunbird-rc',
        'AUTHENTICATION_ENABLED': 'true',
        'CLAIMS_ENABLED': 'true'
    }
    
    def __init__(self):
        self.config = self.DEFAULTS.copy()
        self._load_from_env()
    
    def _load_from_env(self) -> None:
        """Load configuration from environment variables."""
        for key in self.DEFAULTS:
            env_key = key.upper()
            env_value = os.getenv(env_key)
            if env_value:
                self.config[key] = env_value
    
    def get(self, key: str, default: Optional[str] = None) -> str:
        """Get configuration value."""
        return self.config.get(key, default or self.DEFAULTS.get(key, ''))
    
    def is_authentication_enabled(self) -> bool:
        """Check if authentication is enabled."""
        return self.get('AUTHENTICATION_ENABLED').lower() == 'true'
    
    def is_claims_enabled(self) -> bool:
        """Check if claims are enabled."""
        return self.get('CLAIMS_ENABLED').lower() == 'true'
    
    def get_urls(self) -> Dict[str, str]:
        """Get service URLs."""
        return {
            'registry': self.get('REGISTRY_URL'),
            'keycloak': self.get('KEYCLOAK_URL'),
            'claims': self.get('CLAIMS_URL')
        }

# Example configuration
config = SunbirdRCConfig()

print("Configuration:")
print("-" * 50)
print(f"Registry URL: {config.get('REGISTRY_URL')}")
print(f"Keycloak URL: {config.get('KEYCLOAK_URL')}")
print(f"Realm: {config.get('KEYCLOAK_REALM')}")
print(f"Authentication Enabled: {config.is_authentication_enabled()}")
print(f"Claims Enabled: {config.is_claims_enabled()}")
print()
print("Service URLs:")
for service, url in config.get_urls().items():
    print(f"  {service}: {url}")

Configuration:
--------------------------------------------------
Registry URL: http://localhost:8081
Keycloak URL: http://localhost:8080
Realm: sunbird-rc
Authentication Enabled: True
Claims Enabled: True

Service URLs:
  registry: http://localhost:8081
  keycloak: http://localhost:8080
  claims: http://localhost:8082


## Summary

Sunbird RC provides a comprehensive framework for building verifiable digital registries:

Key Components:
1. Authentication via Keycloak with JWT tokens
2. Auto-generated OSID for every entity and nested object
3. Automated claims generation for properties requiring attestation
4. Multi-step attestation workflow with audit trails
5. W3C-compliant verifiable credential generation
6. Public credential verification capability

Workflow:
Entity Creation -> Property Update -> Policy Evaluation -> Claim Generation -> Attestor Review -> Claim Closure -> Credential Generation -> Public Verification

Development:
All API requests require JWT bearer token (except public endpoints)
Every entity receives unique OSID for identification and reference
Claims are automatically created based on attestation policies
Complete audit trail maintained for all operations